In [1]:
import cv2
import pandas as pd
import numpy as np

def preprocessing(val=True):
    
    data = pd.read_csv('data/x_train_gr_smpl.csv')
    labels = pd.read_csv('data/y_train_smpl.csv')

    if(val):
        ## ---------------- Data preparation ---------------- ##
        X_train = []
        for i in range(data.shape[0]):
            img = np.uint8(data.iloc[i])
            edited = cv2.Canny(img, 10, 30)
            edited = cv2.GaussianBlur(edited, (5, 5), 0)
            X_train.append(edited.reshape((1,-1))[0])

        data = pd.DataFrame(X_train)
        ## -------------------------------------------------- ##

    return data, labels

In [2]:
# Point 5: Deeper analysis of the data

top_2_array, top_5_array, top_10_array = set(), set(), set()
bottom_2_array, bottom_5_array, bottom_10_array = set(), set(), set()

data, _ = preprocessing()

for i in range(10):
    data = data.reindex(np.arange(data.shape[0]))
    labels = pd.read_csv(f'data/y_train_smpl_{i}.csv')
    data['label'] = labels
    data = data.sample(frac=1)
    
    corr_label = data.drop("label", axis=1).apply(lambda x: x.corr(data.label))
    corr_label = [(index, abs(corr_val), i) for index, corr_val in enumerate(corr_label)]
    corr_label = sorted(corr_label, key=lambda tup: tup[1], reverse=True)  # Order by correlation value
            
    for i, tup in enumerate(corr_label[:10]):
        if i < 2:
            top_2_array.add(tup[0])
        if i < 5:
            top_5_array.add(tup[0])
        if i < 10:
            top_10_array.add(tup[0])
            
    for i, tup in enumerate(corr_label[-10:]):
        if i < 2:
            bottom_2_array.add(tup[0])
        if i < 5:
            bottom_5_array.add(tup[0])
        if i < 10:
            bottom_10_array.add(tup[0])

In [3]:
# Point 6: Try to improve the classification

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report

data, labels = preprocessing()

In [4]:
data_top_2 = data[data.columns[list(top_2_array)]].copy(deep=True)
data_top_2['label'] = labels

X = data_top_2.iloc[:, :-1]  # All columns but the label
y = data_top_2['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.46      0.53       462
           1       0.54      0.35      0.42       621
           2       0.41      0.58      0.48       157
           3       0.73      0.88      0.80       431
           4       0.86      0.75      0.80       675
           5       0.85      0.76      0.80       717
           6       0.49      0.84      0.62       267
           7       0.23      0.89      0.36        75
           8       0.86      0.82      0.84       668
           9       0.57      0.43      0.49       105

    accuracy                           0.68      4178
   macro avg       0.61      0.67      0.61      4178
weighted avg       0.71      0.68      0.68      4178



In [5]:
data_top_5 = data[data.columns[list(top_5_array)]].copy(deep=True)
data_top_5['label'] = labels

X = data_top_5.iloc[:, :-1]
y = data_top_5['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.57      0.58       482
           1       0.57      0.36      0.44       594
           2       0.40      0.73      0.51       129
           3       0.76      0.91      0.83       442
           4       0.86      0.79      0.83       703
           5       0.87      0.81      0.84       713
           6       0.67      0.86      0.76       247
           7       0.33      0.88      0.49        77
           8       0.90      0.84      0.87       699
           9       0.64      0.78      0.70        92

    accuracy                           0.73      4178
   macro avg       0.66      0.75      0.69      4178
weighted avg       0.75      0.73      0.73      4178



In [6]:
data_top_10 = data[data.columns[list(top_10_array)]].copy(deep=True)
data_top_10['label'] = labels

X = data_top_10.iloc[:, :-1]
y = data_top_10['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

gnb = GaussianNB()

y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.62      0.63       443
           1       0.64      0.44      0.52       612
           2       0.44      0.76      0.55       137
           3       0.76      0.97      0.85       459
           4       0.90      0.86      0.88       715
           5       0.96      0.87      0.91       723
           6       0.80      0.91      0.85       255
           7       0.60      0.90      0.72        78
           8       0.92      0.87      0.89       658
           9       0.61      0.85      0.71        98

    accuracy                           0.79      4178
   macro avg       0.73      0.80      0.75      4178
weighted avg       0.80      0.79      0.79      4178



In [7]:
confusion_matrix(y_test, y_pred)

array([[275,  84,  29,  14,   2,   1,  18,  10,   2,   8],
       [137, 269,  91,  21,  16,   4,  40,   7,   9,  18],
       [ 10,   7, 104,   2,   1,   0,   0,   2,   9,   2],
       [  1,   9,   1, 443,   2,   0,   0,   0,   2,   1],
       [  1,   8,   8,  47, 616,  22,   0,   5,   6,   2],
       [  1,  12,   0,  12,  35, 627,   0,  12,  15,   9],
       [  2,  15,   1,   4,   0,   0, 233,   0,   0,   0],
       [  0,   0,   0,   0,   1,   0,   0,  70,   1,   6],
       [  6,  15,   5,  38,   7,   1,   0,   7, 573,   6],
       [  3,   0,   0,   1,   1,   0,   0,   3,   7,  83]])

# 7. Conclusions (Draft)

### Which streets signs are harder to recognise?
Based on the recall measure labels 0 and 1 are the hardest to recognize.
### Which street signs are most easily confused?
Based on the confusion matrix the most confused signs are the label 0 and 1. 
### Which attributes (fields) are more reliable and which are less reliable in classification of street signs?
By looking at top_2_array we can identify the most reliable features (pixels). By doing some operations like the module we can see those pixels are closer to the centre of the image (or at least don't belong to the borders).
Regarding the less reliable pixels we consider the ones with the lowest correlation value with respect to the label.
### What was the purpose of Tasks 5 and 6?
Prepare the data in a way that we keep the most relevant attributes.
### What would happen if the data sets you used in Tasks 4, 5 and 6 were not randomised?
It doens't happen anything, the results are the same. TODO: Try with preprocessed data.
### What would happen if there is cross-correlation between the non-class attributes?
There is actually cross-correlation between pixels that are close to each other. As a consecuence of that, if two pixels are correlated (adjacent) to each other, and one of them is included in the top 10 of the most correlated pixels with the label, the other is likely to be in the top 10 as well.